In [2]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 50) ## to display more rows
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, ParameterGrid, GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'data-454'
bucket = s3.Bucket(bucket_name)

## Defining the file to be read from s3 bucket
file_key_train = 'Project_1/dmc2010_train.txt'
file_key_test = 'Project_1/dmc2010_class.txt'

bucket_object_train = bucket.Object(file_key_train)
file_object_train = bucket_object_train.get()
file_content_stream_train = file_object_train.get('Body')

bucket_object_test = bucket.Object(file_key_test)
file_object_test = bucket_object_test.get()
file_content_stream_test = file_object_test.get('Body')

## Reading train and test files
train = pd.read_csv(file_content_stream_train, sep = ';')
test = pd.read_csv(file_content_stream_test, sep = ';')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Feature Engineering on Train

In [3]:
train['feature_1'] = np.where((train['salutation'] == 0) & (train['w0'] > 3), 1, 0)
train['feature_2'] = np.where((train['salutation'] == 0) & (train['w1'] > 1), 1, 0)
train['feature_3'] = np.where((train['salutation'] == 0) & (train['w2'] > 0), 1, 0)
train['feature_4'] = np.where((train['salutation'] == 0) & (train['w5'] > 0), 1, 0)
train['feature_5'] = np.where((train['salutation'] == 0) & (train['w9'] > 0), 1, 0)

train['feature_6'] = np.where((train['salutation'] == 1) & (train['w0'] > 2), 1, 0)
train['feature_7'] = np.where((train['salutation'] == 1) & (train['w1'] > 0), 1, 0)
train['feature_8'] = np.where((train['salutation'] == 1) & (train['w2'] > 0), 1, 0)
train['feature_9'] = np.where((train['salutation'] == 1) & (train['w5'] > 0), 1, 0)
train['feature_10'] = np.where((train['salutation'] == 1) & (train['w9'] > 0), 1, 0)

train['feature_11'] = np.where((train['salutation'] == 2) & (train['w0'] > 1), 1, 0)
train['feature_12'] = np.where((train['salutation'] == 2) & (train['w9'] > 0), 1, 0)

train.head()

,customernumber,date,salutation,title,domain,datecreated,newsletter,model,paymenttype,deliverytype,invoicepostcode,delivpostcode,voucher,advertisingdatacode,case,numberitems,gift,entry,points,shippingcosts,deliverydatepromised,deliverydatereal,weight,remi,cancel,used,w0,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,target90,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12
0,41191,2008-12-01,0,0,9,2008-12-01,0,2,2,0,58,NaN,1,NaN,2,2,0,1,0,0,2008-12-03,2008-12-02,737,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,38860,2008-12-16,1,0,4,2008-12-16,0,1,1,1,34,NaN,0,NaN,2,2,0,0,0,0,2008-12-30,2009-02-03,368,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,61917,2008-08-19,0,0,12,2008-08-19,0,1,0,0,51,NaN,0,NaN,1,1,0,0,0,1,2008-09-02,2008-08-28,47,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,40647,2008-06-16,1,0,8,2008-06-16,0,1,0,0,25,NaN,0,NaN,3,2,0,0,0,0,2008-06-17,0000-00-00,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,1347,2008-08-08,0,0,1,2008-08-08,0,1,1,1,41,NaN,0,BR,4,2,0,0,0,0,2008-08-11,2008-08-08,843,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [3]:
pd.crosstab(train['feature_1'], train['target90'])

target90,0,1
feature_1,,
0,18806,4218
1,7571,1833


## Feature Engineering on Test

In [4]:
test['feature_1'] = np.where((test['salutation'] == 0) & (test['w0'] > 3), 1, 0)
test['feature_2'] = np.where((test['salutation'] == 0) & (test['w1'] > 1), 1, 0)
test['feature_3'] = np.where((test['salutation'] == 0) & (test['w2'] > 0), 1, 0)
test['feature_4'] = np.where((test['salutation'] == 0) & (test['w5'] > 0), 1, 0)
test['feature_5'] = np.where((test['salutation'] == 0) & (test['w9'] > 0), 1, 0)

test['feature_6'] = np.where((test['salutation'] == 1) & (test['w0'] > 2), 1, 0)
test['feature_7'] = np.where((test['salutation'] == 1) & (test['w1'] > 0), 1, 0)
test['feature_8'] = np.where((test['salutation'] == 1) & (test['w2'] > 0), 1, 0)
test['feature_9'] = np.where((test['salutation'] == 1) & (test['w5'] > 0), 1, 0)
test['feature_10'] = np.where((test['salutation'] == 1) & (test['w9'] > 0), 1, 0)

test['feature_11'] = np.where((test['salutation'] == 2) & (test['w0'] > 1), 1, 0)
test['feature_12'] = np.where((test['salutation'] == 2) & (test['w9'] > 0), 1, 0)

test.head()

,customernumber,date,salutation,title,domain,datecreated,newsletter,model,paymenttype,deliverytype,invoicepostcode,delivpostcode,voucher,advertisingdatacode,case,numberitems,gift,entry,points,shippingcosts,deliverydatepromised,deliverydatereal,weight,remi,cancel,used,w0,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12
0,6044,2008-07-02,2,0,12,2008-07-02,0,1,2,0,29,NaN,0,NaN,5,1,0,0,0,0,2008-08-01,2008-07-03,3,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,36314,2008-09-09,1,0,4,2008-09-09,0,3,1,1,31,NaN,0,NaN,2,1,0,1,0,0,2008-09-11,2008-09-11,260,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,41932,2008-10-11,0,0,5,2008-10-11,1,1,2,0,50,NaN,1,AX,1,1,0,0,0,0,2008-10-14,0000-00-00,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,3194,2008-09-15,2,0,12,2008-09-15,0,1,0,0,65,NaN,0,NaN,3,1,0,0,0,0,2008-09-16,2008-09-15,719,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,35363,2008-04-02,0,0,0,2008-04-02,0,1,3,0,20,NaN,0,NaN,3,2,0,0,0,0,2008-05-02,2008-04-02,636,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0


In [5]:
test.describe()

,customernumber,salutation,title,domain,newsletter,model,paymenttype,deliverytype,voucher,case,numberitems,gift,entry,points,shippingcosts,weight,remi,cancel,used,w0,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12
count,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000,32427.0,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000,32427.00000,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000,32427.000000
mean,32879.849508,0.541894,0.006507,7.494804,0.165726,1.647485,1.008172,0.205446,0.163136,2.932124,2.010084,0.004317,0.416042,0.0,0.146267,637.414284,0.060690,0.061708,0.067691,0.907731,0.391587,0.29571,0.017948,0.044407,0.178586,0.026891,0.017917,0.000278,0.160946,0.090511,0.020292,0.057421,0.073611,0.061554,0.049958,0.022512,0.058994,0.032812,0.062664,0.032442,0.021124,0.007586
std,19102.529370,0.653451,0.080404,3.678164,0.371840,0.824840,1.090841,0.404033,0.369495,1.312697,1.694554,0.065566,0.492908,0.0,0.353379,719.061675,0.426869,0.336860,0.452057,1.692006,1.227746,1.51476,0.224313,0.363480,0.564006,0.256931,0.193857,0.016658,0.827073,0.616128,0.140999,0.232649,0.261142,0.240347,0.217862,0.148344,0.235617,0.178147,0.242361,0.177174,0.143801,0.086770
min,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,16338.500000,0.000000,0.000000,4.000000,0.000000,1.000000,0.000000,0.000000,0.000000,2.000000,1.000000,0.000000,0.000000,0.0,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,32755.000000,0.000000,0.000000,8.000000,0.000000,1.000000,1.000000,0.000000,0.000000,3.000000,1.000000,0.000000,0.000000,0.0,0.000000,497.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,49399.500000,1.000000,0.000000,11.000000,0.000000,2.000000,2.000000,0.000000,0.000000,4.000000,2.000000,0.000000,1.000000,0.0,0.000000,920.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,66255.000000,2.000000,1.000000,12.000000,1.000000,3.000000,3.000000,1.000000,1.000000,5.000000,45.000000,1.000000,1.000000,0.0,1.000000,16461.000000,25.000000,20.000000,18.000000,110.000000,34.000000,79.00000,13.000000,20.000000,11.000000,21.000000,8.000000,1.000000,36.000000,50.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
